# Loschmidt Plots

Plots for the `recirq.otoc.loschmidt.tilted_sqare_lattice` algorithmic benchmark. See the `analysis-walkthrough.ipynb` notebook for more detail into the functions used to create these plots.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Set up reasonable defaults for figure fonts
import matplotlib
matplotlib.rcParams.update(**{
    'axes.titlesize': 14,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'legend.title_fontsize': 12,
    'figure.figsize': (7, 5),
})

## Load Results

Modify the list of `run_id`s passed to `iterload_dataframes` to load datasets.

In [ ]:
import pandas as pd
import numpy as np

import cirq_google as cg

import recirq.otoc.loschmidt.tilted_square_lattice.analysis as analysis

def iterload_dataframes(run_ids):
    for run_id in run_ids:
        raw_results = cg.ExecutableGroupResultFilesystemRecord.from_json(run_id=run_id).load()
        yield analysis.loschmidt_results_to_dataframe(raw_results)

In [ ]:
df = pd.concat(list(iterload_dataframes([
    'simulated-1',
    # ...
])))
print(len(df), 'rows')
df.head()

## Fit vs. Macrocycle Depth

For each topology, the success probability decays exponentially with respect to random circuit macrocycle depth. The fit parameter `f` is the layer fidelity corresponding to a random single qubit gates and entangling gates between all qubits.

In [ ]:
vs_depth_df, vs_depth_gb_cols = analysis.agg_vs_macrocycle_depth(df)
fit_df, exp_ansatz = analysis.fit_vs_macrocycle_depth(df)
total_df = pd.merge(vs_depth_df, fit_df, on=vs_depth_gb_cols)

In [ ]:
colors = plt.get_cmap('tab10')

for i, row in total_df.iterrows():
    plt.errorbar(
        x=row['macrocycle_depth'],
        y=row['success_probability_mean'],
        yerr=row['success_probability_std'],
        marker='o', capsize=5, ls='',
        color=colors(i),
        label=f'{row["width"]}x{row["height"]} ({row["n_qubits"]}q) {row["processor_str"]}; f={row["f"]:.3f}'
    )
    
    xx = np.linspace(np.min(row['macrocycle_depth']), np.max(row['macrocycle_depth']))
    yy = exp_ansatz(xx, a=row['a'], f=row['f'])
    plt.plot(xx, yy, ls='--', color=colors(i))
    
plt.legend(loc='best')
plt.yscale('log')
plt.xlabel('Macrocycle Depth')
plt.ylabel('Success Probability')
plt.tight_layout()

## Fit vs. Quantum Area

We define a quantity called quantum area (`q_area`) which is the circuit width (i.e. number of qubits) multiplied by its depth. This is the number of operations in the circuit (also including any idle operations). The fit parameter `f` is the per-operation, per-qubit fidelity.

In [ ]:
vs_q_area_df, vs_q_area_gb_cols = analysis.agg_vs_q_area(df)
fit_df2, exp_ansatz_vs_q_area = analysis.fit_vs_q_area(df)
total_df2 = pd.merge(vs_q_area_df, fit_df2, on=vs_q_area_gb_cols)

In [ ]:
colors = plt.get_cmap('tab10')

for i, row in total_df2.iterrows():
    plt.errorbar(x=row['q_area'], 
                 y=row['success_probability_mean'], 
                 yerr=row['success_probability_std'],
                 color=colors(i), capsize=5, marker='o', ls='')
    
    xx = np.linspace(np.min(row['q_area']), np.max(row['q_area']))
    yy = exp_ansatz_vs_q_area(xx, a=row['a'], f=row['f'])
    plt.plot(xx, yy, ls='--', color=colors(i),
             label=f'{row["run_id"]}; f={row["f"]:.3f}'
            )


plt.legend(loc='best')
plt.xlabel('Quantum Area')
plt.ylabel('Macrocycle Fidelity')
plt.yscale('log')
plt.tight_layout()